# Training Data Debug

In [3]:
import math
import sys
import time

import matplotlib.pyplot as plt
import numpy as np
import open3d as o3d
from scipy import spatial
from matplotlib import cm

from tqdm.auto import tqdm

from sphere import Sphere
from dh_grid import DHGrid
    
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load the previous (old) data format

In [9]:
export_ds = '/media/berlukas/Data/data/nuscenes/'

cloud_filename = f"{export_ds}/sem_clouds_decoded.npy"
cloud_features_decoded = np.load(cloud_filename)
# cloud_features_decoded = np.argmax(cloud_features_decoded, axis=1)

print(f"Shape of clouds is {cloud_features_decoded.shape}")

Shape of clouds is (200, 200, 200)


# Load the current data format

In [64]:
export_ds = '/media/berlukas/Data/data/nuscenes/'

# cloud_filename = f"{export_ds}/sem_clouds_08.npy"
cloud_filename = f"{export_ds}/sem_clouds_val_kitti_final.npy"
# cloud_filename = f"{export_ds}/sem_clouds_decoded.npy"

print(f"Loading clouds from {cloud_filename}")
cloud_features = np.load(cloud_filename)
print(f"Initial shape of clouds is {cloud_features.shape}")
sem_cloud_features = cloud_features[:, 2, :, :]
cloud_features = cloud_features[:, 0:2, :, :]

print(f"Shape of clouds is {cloud_features.shape}")
# print(f"Shape of clouds is {cloud_features.shape} and sem clouds is {sem_cloud_features.shape}")

Loading clouds from /media/berlukas/Data/data/nuscenes//sem_clouds_val_kitti_final.npy
Initial shape of clouds is (200, 3, 200, 200)
Shape of clouds is (200, 2, 200, 200)


## Compare the two datasets

In [5]:
def mapIntensityToRGB(i):
    mask = np.where(i < 0)    
    colors = cm.jet(plt.Normalize(min(i), max(i))(i))
    colors[mask] = 0
    return colors

def prepare_for_viz(cloud):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(cloud[:, 0:3])
    if cloud.shape[1] == 4:
        colors = mapIntensityToRGB(cloud[:, 3])
        pcd.colors = o3d.utility.Vector3dVector(colors[:,0:3])
    if cloud.shape[1] == 6:
        pcd.colors = o3d.utility.Vector3dVector(cloud[:,3:6] / 255.0)
    return pcd

def visualize_pointcloud(cloud):
    pcd = prepare_for_viz(cloud)        
    o3d.visualization.draw_geometries([pcd])
    
def visualize_sphere(cloud, ndim = 1, bw = 100, idx = 0):
    _, points = DHGrid.CreateGrid(bw)
    cur_sem_cloud = np.reshape(cloud, (ndim, -1)).T        
    
    points_xyz = DHGrid.ConvertPointsToEuclidean(points)                    
    points_xyzi = np.column_stack((points_xyz, cur_sem_cloud[:,idx]))    
    visualize_pointcloud(points_xyzi)
    return points_xyzi
    
def writeRawPointCloud(cloud, filename):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(cloud[:, 0:3])
    colors = mapIntensityToRGB(cloud[:, 3])
    pcd.colors = o3d.utility.Vector3dVector(colors[:,0:3])
    o3d.io.write_point_cloud(filename, pcd)
    

In [67]:
xyzi = visualize_sphere(cloud_features[150,:,:,:], 2, 100, 1)

In [10]:
visualize_sphere(cloud_features_decoded[150,:,:], 1, 100, 0)

array([[ 7.85390089e-03,  0.00000000e+00,  9.99969158e-01,
         0.00000000e+00],
       [ 7.85002546e-03,  2.46696989e-04,  9.99969158e-01,
         0.00000000e+00],
       [ 7.83840301e-03,  4.93150517e-04,  9.99969158e-01,
         0.00000000e+00],
       ...,
       [ 7.81904500e-03, -7.39117366e-04, -9.99969158e-01,
         0.00000000e+00],
       [ 7.83840301e-03, -4.93150517e-04, -9.99969158e-01,
         0.00000000e+00],
       [ 7.85002546e-03, -2.46696989e-04, -9.99969158e-01,
         0.00000000e+00]])